In [1]:
import pandas as pd 
import os
import numpy as np

COMPSEG_FILE = r'data\firm_info\compustats_segment_00_21.dta'
HH_FILE = r"data\hh_dataset\publicfirm_site.dta"

industry_map = {
    range(1, 10): "AG-M-C",
    range(10, 15): "AG-M-C",
    range(15, 18): "AG-M-C",
    range(20, 40): "MANUF",
    range(40, 50): "TR-UTL",
    range(50, 52): "WHL-RT",
    range(52, 60): "WHL-RT",
    range(60, 68): "F-I-RE",
    range(70, 90): "SVCS",
    range(91, 100): "GOVT"
}

def filter_high_priority(df):
    min_priority = df.groupby(['gvkey', 'fyear'])['priority'].transform('min')
    return df[df['priority'] == min_priority]


def map_industry(code):
    for code_range, abbr in industry_map.items():
        if code in code_range:
            return abbr
    return None

In [3]:
cd C:\Users\ttshuang\Dropbox\Phil\phil-lisa-work

C:\Users\ttshuang\Dropbox\Phil\phil-lisa-work


In [41]:
# read data and keep relevant columns
comp_seg = pd.read_stata(COMPSEG_FILE)[
    ['gvkey','fyear','total_sales','sid','stype','snms','emps','seg_sale','SICS1']
    ].sort_values(['gvkey','fyear']).reset_index(drop=True)

# Only keep year 2001 to 2009 as the HH data is from 2001 to 2009
comp_seg = comp_seg[(comp_seg['fyear']<=2009) & (comp_seg['fyear']>=2001)].reset_index(drop=True)
# only keep one stype based on the priority
priority = {'GEOSEG': 2, 'BUSSEG': 2, 'OPSEG': 3, 'STSEG': 1}
comp_seg['priority'] = comp_seg['stype'].map(priority)
comp_seg = filter_high_priority(comp_seg)
comp_seg = comp_seg.drop(columns=['priority'])

In [42]:
comp_seg.head(10)

,gvkey,fyear,total_sales,sid,stype,snms,emps,seg_sale,SICS1
0,001004,2001.0,638.721,12.0,BUSSEG,Inventory and Logistic Services,NaN,258.067,5088
1,001004,2001.0,638.721,13.0,BUSSEG,"Maintenance, Repair and Overhaul",NaN,216.727,4581
2,001004,2001.0,638.721,10.0,BUSSEG,Manufacturing,NaN,99.558,3559
3,001004,2001.0,638.721,14.0,BUSSEG,Aircraft and Engine Sales and Leasing,NaN,64.369,5088
4,001004,2002.0,606.337,16.0,BUSSEG,Structures and Systems,NaN,130.628,3559
5,001004,2002.0,606.337,15.0,BUSSEG,Aviation Supply Chain,NaN,358.412,5088
6,001004,2002.0,606.337,17.0,BUSSEG,Aircraft Sales and Leasing,NaN,17.387,5088
7,001004,2002.0,606.337,18.0,BUSSEG,"Maintenance, Repair and Overhaul",NaN,93.415,4581
8,001004,2003.0,651.958,15.0,BUSSEG,Aviation Supply Chain,NaN,349.527,5088
9,001004,2003.0,651.958,17.0,BUSSEG,Aircraft Sales and Leasing,NaN,24.969,5088


In [43]:
# read processed hh_site data
hh_site = pd.read_stata(HH_FILE)[
    ['gvkey','year','corpid','siteid','emple','reven','avg_reven']
    ].sort_values(['gvkey','year']).reset_index(drop=True)
hh_site.head(10)

,gvkey,year,corpid,siteid,emple,reven,avg_reven
0,001004,2001.0,5689,110023187,99,14,0.141414
1,001004,2001.0,5689,107007880,104,18,0.173077
2,001004,2001.0,5689,114001316,400,344,0.860000
3,001004,2001.0,5689,123000762,150,70,0.466667
4,001004,2001.0,5689,110018783,290,60,0.206897
5,001004,2002.0,5689,114001316,400,344,0.860000
6,001004,2002.0,5689,123000762,90,42,0.466667
7,001004,2002.0,5689,110023187,130,18,0.138462
8,001004,2002.0,5689,107007880,60,11,0.183333
9,001004,2002.0,5689,110018783,200,42,0.210000


In [44]:
# No industry for this processed industry data - add 2004 data from the raw data
# start from raw hh 2004 data - add sic code 
raw_2004 = pd.read_csv(r'data\hh_dataset\USA_2004\Hist2004_SITEDESC.txt', delimiter="\t", encoding='latin1')
cols = ['SITEID','CORPHDQ','SICCODE','SICGRP','ENTREVENUE']
raw_2004 = raw_2004[cols]
raw_2004.columns = ['siteid','corphdq','siccode','SICGRP','entrevenue']

# only look at 2004 data
hh_site_2004 = hh_site[hh_site['year']==2004]
# link with hh_site 2004 data
hh_site_2004 = hh_site_2004.merge(raw_2004, on='siteid', how='left' )

# map ind group for compustats data
comp_seg_2004 = comp_seg[comp_seg['fyear']==2004]
comp_seg_2004['sic2d'] = comp_seg_2004['SICS1'].astype(str).str[:2].astype(int) 
comp_seg_2004['SICGRP'] = comp_seg_2004['sic2d'].apply(map_industry)

C:\Users\ttshuang\AppData\Local\Temp\ipykernel_22684\2869924936.py:3: DtypeWarning: Columns (6,29,35,87) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_2004 = pd.read_csv(r'data\hh_dataset\USA_2004\Hist2004_SITEDESC.txt', delimiter="\t", encoding='latin1')
C:\Users\ttshuang\AppData\Local\Temp\ipykernel_22684\2869924936.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  comp_seg_2004['sic2d'] = comp_seg_2004['SICS1'].astype(str).str[:2].astype(int)
C:\Users\ttshuang\AppData\Local\Temp\ipykernel_22684\2869924936.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan

In [51]:
comp_seg_2004_geo = comp_seg_2004[comp_seg_2004['stype']=='GEOSEG']
comp_seg_2004_geo[:20]

,gvkey,fyear,total_sales,sid,stype,snms,emps,seg_sale,SICS1,sic2d,SICGRP
79,001021,2004.0,19.833,1.0,GEOSEG,United States,NaN,19.833,3861,38,MANUF
104,001034,2004.0,1339.480,11.0,GEOSEG,Other,NaN,102.029,2834,28,MANUF
105,001034,2004.0,1339.480,8.0,GEOSEG,Norway,NaN,18.600,2834,28,MANUF
107,001034,2004.0,1339.480,1.0,GEOSEG,United States,NaN,368.400,2834,28,MANUF
110,001034,2004.0,1339.480,9.0,GEOSEG,Denmark,NaN,24.300,2834,28,MANUF
155,001045,2004.0,18645.000,7.0,GEOSEG,Latin America,NaN,3115.000,4512,45,TR-UTL
156,001045,2004.0,18645.000,2.0,GEOSEG,United States,NaN,12192.000,4512,45,TR-UTL
157,001045,2004.0,18645.000,8.0,GEOSEG,Pacific,NaN,660.000,4512,45,TR-UTL
158,001045,2004.0,18645.000,5.0,GEOSEG,Europe,NaN,2678.000,4512,45,TR-UTL
211,001056,2004.0,414.101,2.0,GEOSEG,United States,2398.0,414.101,3825,38,MANUF


In [53]:
hh_site_2004_geo = hh_site_2004[hh_site_2004['gvkey'].isin(comp_seg_2004_geo.gvkey.tolist())]
hh_site_2004_geo[:20]

,gvkey,year,corpid,siteid,emple,reven,avg_reven,corphdq,siccode,SICGRP,entrevenue
13,001021,2004.0,5278,147007555,60,8,0.133333,BRANCH,781,AG-M-C,24
14,001021,2004.0,5278,121024812,35,3,0.085714,BRANCH,8661,SVCS,24
15,001021,2004.0,5278,147002506,4,1,0.250000,BRANCH,3861,MANUF,24
16,001021,2004.0,5278,133015300,200,44,0.220000,ULTIMATE HQ,3843,MANUF,24
17,001075,2004.0,3050,103005467,1000,256,0.256000,ULTIMATE HQ,4911,TR-UTL,2834
18,001075,2004.0,3050,111012308,100,10,0.100000,BRANCH,8999,SVCS,2834
19,001075,2004.0,3050,103015808,15,1,0.066667,BRANCH,7011,SVCS,2834
20,001075,2004.0,3050,103027250,19,5,0.263158,BRANCH,4911,TR-UTL,2834
21,001075,2004.0,3050,103006061,250,64,0.256000,BRANCH,4911,TR-UTL,2834
22,001075,2004.0,3050,103005957,70,9,0.128571,DIVISIONAL HQ,6552,F-I-RE,2834


In [55]:
comp_seg_2004_st = comp_seg_2004[comp_seg_2004['stype']=='STSEG']
comp_seg_2004_st[:50]

,gvkey,fyear,total_sales,sid,stype,snms,emps,seg_sale,SICS1,sic2d,SICGRP
45343,012125,2004.0,NaN,4.0,STSEG,Southern California,NaN,15.873,6798,67,F-I-RE
45344,012125,2004.0,NaN,12.0,STSEG,Arizona,NaN,18.614,6798,67,F-I-RE
45345,012125,2004.0,NaN,5.0,STSEG,Northwest,NaN,9.655,6798,67,F-I-RE
45346,012125,2004.0,NaN,11.0,STSEG,Colorado,NaN,13.924,6798,67,F-I-RE
51525,013562,2004.0,NaN,11.0,STSEG,Corporate,NaN,7.626,6798,67,F-I-RE
51526,013562,2004.0,NaN,10.0,STSEG,Urban,NaN,26.319,6798,67,F-I-RE
51527,013562,2004.0,NaN,9.0,STSEG,Pennsylvania - North,NaN,73.882,6798,67,F-I-RE
51528,013562,2004.0,NaN,8.0,STSEG,Pennsylvania - West,NaN,87.535,6798,67,F-I-RE
51529,013562,2004.0,NaN,7.0,STSEG,New Jersey,NaN,94.096,6798,67,F-I-RE
51530,013562,2004.0,NaN,5.0,STSEG,Virginia,NaN,27.099,6798,67,F-I-RE


In [54]:
hh_site_2004_st = hh_site_2004[hh_site_2004['gvkey'].isin(comp_seg_2004_st.gvkey.tolist())]
hh_site_2004_st[:20]

,gvkey,year,corpid,siteid,emple,reven,avg_reven,corphdq,siccode,SICGRP,entrevenue
38525,014790,2004.0,734,103006159,25,17,0.680000,NOT LINKED,1521,AG-M-C,17
38526,014790,2004.0,734,103031232,25,17,0.680000,ULTIMATE HQ,1521,AG-M-C,1471
38527,014790,2004.0,734,144024497,60,30,0.500000,DIVISIONAL HQ,1542,AG-M-C,1471
42095,020021,2004.0,6872,115008923,1200,54,0.045000,BRANCH,7011,SVCS,834
42096,020021,2004.0,6872,103006289,21,1,0.047619,ULTIMATE HQ,7999,SVCS,834
42097,020021,2004.0,6872,129000838,2500,185,0.074000,BRANCH,7011,SVCS,834
42098,020021,2004.0,6872,126006045,325,20,0.061538,BRANCH,7993,SVCS,834
42099,020021,2004.0,6872,129001722,1450,107,0.073793,BRANCH,7011,SVCS,834
42100,020021,2004.0,6872,131005029,5000,460,0.092000,BRANCH,7011,SVCS,834
47826,030293,2004.0,5577,105030518,60,27,0.450000,ULTIMATE HQ,6798,F-I-RE,234


In [56]:
temp1 = comp_seg_2004[comp_seg_2004['gvkey']=='001004']
temp2 = hh_site_2004[hh_site_2004['gvkey']=='001004']

In [60]:
temp1.sort_values('SICGRP')

,gvkey,fyear,total_sales,sid,stype,snms,emps,seg_sale,SICS1,sic2d,SICGRP
15,001004,2004.0,747.848,16.0,BUSSEG,Structures and Systems,NaN,193.296,3559,35,MANUF
14,001004,2004.0,747.848,18.0,BUSSEG,"Maintenance, Repair and Overhaul",NaN,111.932,4581,45,TR-UTL
12,001004,2004.0,747.848,15.0,BUSSEG,Aviation Supply Chain,NaN,390.060,5088,50,WHL-RT
13,001004,2004.0,747.848,17.0,BUSSEG,Aircraft Sales and Leasing,NaN,45.139,5088,50,WHL-RT


In [61]:
temp2.sort_values('SICGRP')

,gvkey,year,corpid,siteid,emple,reven,avg_reven,corphdq,siccode,SICGRP,entrevenue
1,001004,2004.0,5689,110023187,130,18,0.138462,BRANCH,3728,MANUF,854
4,001004,2004.0,5689,123000762,105,49,0.466667,BRANCH,3728,MANUF,854
7,001004,2004.0,5689,107007880,60,11,0.183333,BRANCH,3511,MANUF,854
8,001004,2004.0,5689,123009340,450,39,0.086667,BRANCH,2448,MANUF,854
3,001004,2004.0,5689,133034746,70,9,0.128571,BRANCH,7699,SVCS,854
6,001004,2004.0,5689,133018882,170,22,0.129412,BRANCH,7699,SVCS,854
0,001004,2004.0,5689,133011193,60,11,0.183333,BRANCH,4581,TR-UTL,854
2,001004,2004.0,5689,110018783,200,42,0.210000,BRANCH,4581,TR-UTL,854
5,001004,2004.0,5689,114001316,350,301,0.860000,ULTIMATE HQ,5063,WHL-RT,854


In [62]:
temp1.groupby('SICGRP').agg({"seg_sale": 'sum'})

,seg_sale
SICGRP,
MANUF,193.296
TR-UTL,111.932
WHL-RT,435.199


In [63]:
temp2.groupby('SICGRP').agg({"reven": 'sum'})

,reven
SICGRP,
MANUF,117
SVCS,31
TR-UTL,53
WHL-RT,301
